In [29]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools
import math
import os
import sympy as sp
import random
from sklearn.model_selection import train_test_split
from sympy import symbols, Function, diff
output_directory = r'../../../Data/3D_Lotka_Volterra/3D_Lotka_T5_N10'
def generate_random_values():
    x1 = random.uniform(0.05, 2)
    x2 = random.uniform(0.05, 2)
    x3 = random.uniform(0.05, 2)
    return x1, x2, x3
def calculate_c(x1, x2, x3):
    c = x1 + x2 + x3
    return c
def generate_random_values_based_on_c(c):
    min_val = 0.05
    max_val = 2
    x1 = random.uniform(min_val, min(max_val, c - min_val * 2))
    x2 = random.uniform(min_val, min(max_val, c - x1 - min_val))
    x3 = c - x1 - x2
    return x1, x2, x3
def generate_data(initial_conditions):
    def normalize(vector):
        norm = np.linalg.norm(vector)
        if norm == 0: 
            return vector
        return vector / norm
    def normalized_system(y, t):
        x1, x2, x3 = y
        f = np.array([x1 * (x2 - x3), x2 * (x3 - x1), x3 * (x1 - x2)])
        normalized_f = normalize(f)
        return normalized_f
    num_trajectories = 5
    t = np.linspace(0, 100, 10) # 10 data points per trajectory
    all_trajectory_data = []
    initial_conditions_to_print = []
    print("Initial data (x1, x2, x3):")
    for i, initial_condition in enumerate(initial_conditions):
        print(f"({initial_condition[0]}, {initial_condition[1]}, {initial_condition[2]})")
        sol = odeint(normalized_system, initial_condition, t)
        all_trajectory_data.append(sol)
    num_variables = 3 # Adjust number of variables that we need for the regression accordingly
    column_names = [f'x{i+1}' for i in range(num_variables)]
    column_names.append('trajectory')
    with open('50.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for r, data in enumerate(all_trajectory_data):
            for j in range(len(t)):
                row = data[j].tolist() + [r + 1]
                writer.writerow(row) 
    output_directory1 = r'../../../results/3D_Lotka_Volterra/3D_Lotka_T5_N10'
    plt.figure(figsize=(10, 6))
    for i, sol in enumerate(all_trajectory_data):
        for j in range(sol.shape[1]):
            plt.plot(t, sol[:, j])
    plt.savefig(os.path.join(output_directory1, 'trajectory.png'))
    plt.close()
def split_data():
    trajectories = {}
    column_names = None
    with open(r'../../../Data/3D_Lotka_Volterra/3D_Lotka_T5_N10/trainingp_data50.csv', 'r') as trainfile:
        reader = csv.DictReader(trainfile)
        column_names = reader.fieldnames
        for row in reader:
            trajectory = float(row['trajectory'])
            if trajectory not in trajectories:
                trajectories[trajectory] = []
            trajectory_data = {key: float(value) for key, value in row.items()}
            trajectories[trajectory].append(trajectory_data)
    for traj_points in trajectories.values():
        random.shuffle(traj_points)
    num_points_per_file = len(next(iter(trajectories.values()))) // 5  # divide into five splits (n stratify)
    for i in range(5):  # Five-fold cross-validation
        output_filename = f'B50{i+1}.csv'
        with open(os.path.join(output_directory, output_filename), 'w', newline='') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=column_names)
            writer.writeheader()
            for trajectory, points in trajectories.items():
                for point in points[i * num_points_per_file: (i + 1) * num_points_per_file]:
                    writer.writerow(point)
if __name__ == "__main__":
    x1, x2, x3 = generate_random_values()
    c = calculate_c(x1, x2, x3)
    initial_conditions = [generate_random_values_based_on_c(c) for _ in range(5)]
    generate_data(initial_conditions)
    data = np.genfromtxt('50.csv', delimiter=',', names=True)
    training_data = []
    holdout_data = []
    for r in range(1, 6):  # this represents the number of initial data is 5. i.e., (1,6) means 5 initial data
        trajectory_subset = data[data['trajectory'] == r]
        train_set, holdout_set = train_test_split(trajectory_subset, test_size=0.2, random_state=42)
        training_data.extend(train_set)
        holdout_data.extend(holdout_set)
    column_names = data.dtype.names
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'w', newline='') as trainfile:
        writer = csv.writer(trainfile)
        writer.writerow(column_names)
        for row in training_data:
            writer.writerow([row[col] for col in column_names])
    with open(os.path.join(output_directory, 'holdoutp_data50.csv'), 'w', newline='') as holdfile:
        writer = csv.writer(holdfile)
        writer.writerow(column_names)
        for row in holdout_data:
            writer.writerow([row[col] for col in column_names])
    split_data()

Initial data (x1, x2, x3):
(0.9911737093494568, 1.523416263304777, 0.7851207660494588)
(1.9556349709221001, 0.6879000697274102, 0.6561756980541822)
(0.6002620400313977, 0.08252326988293676, 2.6169254287893584)
(1.7497033906553365, 0.24368740511830045, 1.3063199429300556)
(1.2571418904872391, 0.17113748176273852, 1.8714313664537152)


In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import pandas as pd
import os
import random
from sklearn.model_selection import train_test_split
output_directory = r'../../../Data/3D_Lotka_Volterra/3D_Lotka_T5_N10'
output_director = r'../../../System/3D_Lotka_Volterra/3D_Lotka_T5_N10'
output_directoryk = r'../../../Data/3D_Lotka_Volterra/3D_Lotka_T5_N10/k_fold'
m = 5
m1 = 5  
m2 = 6 
tm = 10
def generate_random_values():
    x1 = random.uniform(0.05, 2)
    x2 = random.uniform(0.05, 2)
    x3 = random.uniform(0.05, 2)
    return x1, x2, x3
def calculate_c(x1, x2, x3):
    c = x1 + x2 + x3
    return c
def generate_random_values_based_on_c(c):
    min_val = 0.05
    max_val = 2
    x1 = random.uniform(min_val, min(max_val, c - min_val * 2))
    x2 = random.uniform(min_val, min(max_val, c - x1 - min_val))
    x3 = c - x1 - x2
    return x1, x2, x3
def generate_data(initial_conditions):
    def normalize(vector):
        norm = np.linalg.norm(vector)
        if norm == 0: 
            return vector
        return vector / norm
    def normalized_system(y, t):
        x1, x2, x3 = y
        f = np.array([x1 * (x2 - x3), x2 * (x3 - x1), x3 * (x1 - x2)])
        normalized_f = normalize(f)
        return normalized_f
    t1 = np.linspace(0, 10, tm)  # forward
    t2 = np.linspace(0, 10, tm)  # backward
    forward_trajectories = []
    backward_trajectories = []
    print("Initial data (x1, x2, x3):")
    for initial_condition in initial_conditions:
        # Print the initial conditions with the first five digits
        formatted_initial_condition = tuple(f"{value:.5f}" for value in initial_condition)
        print(formatted_initial_condition) 
        forward_trajectory = odeint(normalized_system, initial_condition, t1)
        backward_trajectory = odeint(normalized_system, initial_condition, -t2)
        forward_trajectories.append(forward_trajectory[:m1])
        backward_trajectories.append(backward_trajectory[:m2])
    return forward_trajectories, backward_trajectories, t1[:m1], t2[:m2]
def save_data(forward_trajectories, backward_trajectories, t1, t2):
    num_variables = 3 
    column_names = [f'x{i+1}' for i in range(num_variables)]
    column_names.append('trajectory')    
    with open(os.path.join(output_director, '50.csv'), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for r, (forward_data, backward_data) in enumerate(zip(forward_trajectories, backward_trajectories)):
            for j in range(len(t1)):
                forward_row = [f"{value:.5f}" for value in forward_data[j]] + [r + 1]
                writer.writerow(forward_row)
            for j in range(1, len(t2)):  
                backward_row = [f"{value:.3f}" for value in backward_data[j]] + [r + 1]
                writer.writerow(backward_row)
    output_directory1 = r'../../../results/3D_Lotka_Volterra/3D_Lotka_T5_N10'
    plt.figure(figsize=(10, 6))
    for i, (forward_sol, backward_sol) in enumerate(zip(forward_trajectories, backward_trajectories)):
        for j in range(forward_sol.shape[1]):
            plt.plot(t1, forward_sol[:, j])
            plt.plot(-t2, backward_sol[:, j])  
    plt.savefig(os.path.join(output_directory1, 'trajectory.png'))
    plt.close()
def split_data():
    trajectories = {}
    column_names = None
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'r') as trainfile:
        reader = csv.DictReader(trainfile)
        column_names = reader.fieldnames
        for row in reader:
            trajectory = float(row['trajectory'])
            if trajectory not in trajectories:
                trajectories[trajectory] = []
            trajectory_data = {key: float(value) for key, value in row.items()}
            trajectories[trajectory].append(trajectory_data)
    for traj_points in trajectories.values():
        random.shuffle(traj_points)
    num_points_per_file = len(next(iter(trajectories.values()))) // 5  # divide into five splits (n stratify)
    for i in range(5):  # Five-fold cross-validation
        output_filename = f'B50{i + 1}.csv'
        with open(os.path.join(output_directory, output_filename), 'w', newline='') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=column_names)
            writer.writeheader()
            for trajectory, points in trajectories.items():
                for point in points[i * num_points_per_file: (i + 1) * num_points_per_file]:
                    writer.writerow(point)
def k_fold_csv(input_directory, file_indices, output_index, output_directory):
    os.makedirs(output_directory, exist_ok=True)    
    output_file = os.path.join(output_directory, f'a50{output_index}.csv')
    input_files = [f'B50{i}.csv' for i in file_indices]
    combined_df = pd.DataFrame()

    for file in input_files:
        file_path = os.path.join(input_directory, file)
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
        else:
            print(f"File {file_path} does not exist.")   
    combined_df = combined_df.sort_values(by='trajectory')
    combined_df.to_csv(output_file, index=False)
if __name__ == "__main__":
    x1, x2, x3 = generate_random_values()
    c = calculate_c(x1, x2, x3)
    initial_conditions = [generate_random_values_based_on_c(c) for _ in range(m)]
    forward_trajectories, backward_trajectories, t1, t2 = generate_data(initial_conditions)
    save_data(forward_trajectories, backward_trajectories, t1, t2)    
    data = np.genfromtxt(os.path.join(output_director, '50.csv'), delimiter=',', names=True)
    training_data = []
    holdout_data = []
    for r in range(1, 6):  # this represents the number of initial data is 5. i.e., (1,6) means 5 initial data
        trajectory_subset = data[data['trajectory'] == r]
        train_set, holdout_set = train_test_split(trajectory_subset, test_size=0.2, random_state=42)
        training_data.extend(train_set)
        holdout_data.extend(holdout_set)
    column_names = data.dtype.names
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'w', newline='') as trainfile:
        writer = csv.writer(trainfile)
        writer.writerow(column_names)
        for row in training_data:
            writer.writerow([f"{row[col]:.5f}" if col != 'trajectory' else int(row[col]) for col in column_names])
    with open(os.path.join(output_directory, 'holdoutp_data50.csv'), 'w', newline='') as holdfile:
        writer = csv.writer(holdfile)
        writer.writerow(column_names)
        for row in holdout_data:
            writer.writerow([f"{row[col]:.5f}" if col != 'trajectory' else int(row[col]) for col in column_names])
    split_data()
    file_indices_list = [
        [1, 2, 3, 4],
        [1, 2, 3, 5],
        [1, 2, 4, 5],
        [1, 3, 4, 5],
        [2, 3, 4, 5]
    ]
    for i, file_indices in enumerate(file_indices_list, start=1):
        k_fold_csv(output_directory, file_indices, i, output_directoryk)